In [ ]:
import dash_bootstrap_components as dbc
from dash import dcc, html
import uitoolkit_plotly_dash as uitk  # your custom UI toolkit

def get_conversations_html(name):
    return html.Div(
        children=[
            dbc.Select(
                id="llm-type" + name,
                value='gemini-1.5-pro-text',
                options=[
                    {'label': 'Gemini Flash', 'value': 'gemini-1.5-flash-text'},
                    {'label': 'Gemini Pro', 'value': 'gemini-1.5-pro-text'}
                ],
            ),
            html.Div(id="display-conversation" + name),
        ],
        style={
            # Force vertical scrolling if conversation becomes large
            "overflow": "auto",
            "height": "100%",  # fill parent height
            "backgroundColor": "#eeeeee",
            "padding": "10px"
        }
    )

def get_controls_html(name):
    return dbc.InputGroup(
        children=[
            dbc.Input(id="chat-input" + name, placeholder="Message", type="text"),
            dbc.Button("Submit", id="submit" + name),
        ],
        style={"marginTop": "10px", "width": "100%"}
    )

def setup_layout(name, ENV="DEV"):
    """
    Creates a stable two-column layout using:
      - A header at the top
      - Left col for file upload and PDF tabs
      - Right col for conversation
      - Both columns scroll independently
    """

    header_height = 60  # or 70, 80, etc. depending on your header design

    return html.Div(
        style={
            'width': '100%',
            'height': '100vh',
            'margin': 0,
            'padding': 0,
            'overflow': 'hidden'
        },
        children=[
            # Your top header
            uitk.HeaderAllIn(
                menu_items=[{'type': 'outlined', 'id': f"unique_id{name}"}],
                brand={'appName': f'{name} BYOD', 'envBadge': {'name': f'{ENV}'}},
                search_expand_mode='none'
            ),

            # The main content below the header
            html.Div(
                id='page-load-div' + name,
                style={'display': 'none'}
            ),

            # Now a row that fills remaining height
            # We subtract the header_height from full viewport height
            dbc.Row(
                style={
                    "height": f"calc(100vh - {header_height}px)",
                    "margin": "0",
                    "padding": "0"
                },
                children=[
                    # LEFT COLUMN
                    dbc.Col(
                        width=4,  # ~ 33% of the row
                        style={
                            "overflowY": "auto",
                            "borderRight": "1px solid #ddd",  # optional visual divider
                            "padding": "10px"
                        },
                        children=[
                            dcc.Upload(
                                id='upload-data' + name,
                                children=html.Div([
                                    'Drag and Drop or ',
                                    html.A('Select Files (.txt, .pdf)')
                                ]),
                                style={
                                    'width': '100%',
                                    'height': '60px',
                                    'lineHeight': '60px',
                                    'borderWidth': '1px',
                                    'borderStyle': 'dashed',
                                    'borderRadius': '5px',
                                    'textAlign': 'center',
                                    'margin': '10px 0'
                                },
                                multiple=True
                            ),
                            uitk.DynamicTabs(
                                id="file_tabs" + name,
                                active_tab="tab1" + name,
                                tabs=[{}]
                            )
                        ]
                    ),

                    # RIGHT COLUMN
                    dbc.Col(
                        width=8,  # ~ 67% of the row
                        style={
                            "display": "flex",
                            "flexDirection": "column",
                            "height": "100%",
                            "overflow": "hidden",  # We'll manage sub-components' scroll
                            "padding": "10px"
                        },
                        children=[
                            # We'll contain conversation and controls in separate sections
                            # with their own scroll if needed
                            dcc.Store(id="store-conversation" + name, data=""),

                            # Container or Div for the chat area
                            html.Div(
                                id="conversation-container-" + name,
                                style={
                                    "flex": "1",  # fill remaining vertical space
                                    "overflowY": "auto",
                                    "border": "1px solid #ccc",  # optional
                                    "borderRadius": "5px",
                                    "marginBottom": "10px"
                                },
                                children=[
                                    get_conversations_html(name)
                                ]
                            ),

                            # Chat input row
                            get_controls_html(name),

                            # Spinner for LLM loading
                            dbc.Spinner(
                                html.Div(id="loading-component" + name),
                                size="sm"
                            )
                        ]
                    )
                ]
            ),

            # Hidden or background divs
            html.Div(
                id='hidden-div' + name,
                style={'display': 'none'}
            ),

            # If you store files in memory or want to keep track
            uitk.Spinner(
                html.Div(
                    id='storing-file' + name,
                    children=[dcc.Store(id='uploaded-file' + name)],
                    style={'minHeight': '50px', 'display': 'none'}
                )
            )
        ]
    )